(last update final exam Biomol interact, May 31st 2021)

This is simple system to generate specific tests for the students. You need two files: 
* `courseid_22740_participants.csv`: a CSV file containing the names and e-mails of the students (obtained from Moodle): the columns should be named `First name`, `Surname` and `Email address`
* `test.csv`: a CSV file containinf the questions, in the following format:

Points  |  Question   |  data1 

within the`Question` you may use the replacement tag(s) `XXX`(`YYY`,`ZZZ` and other tags), that will be substituted by the info that is under `data1`. It is straighforward to add new tags to substitute.

Improvements needed:
* files are created in a given folder, assuming google drive, and links are created in a  file called `links.txt`. It should be possible to generate a list of e-mails for each file without expecting the students to ask for permissions to check their files.


In [ ]:
import pandas as pd
from unidecode import unidecode as ud

#
# this is the path for all files and folders. It is set up to deal with 
# Google stream, as the different folders are assigned to each student,
# but of course it can be moved to any other folder
# Note on using Google stream: first time the files with the exams do not exist, which implies
#       that the 'xattr' instruction below does not get the correct ID for the generated files
#       and assign them local IDs instead of google drive's. Run the script twice to ensure the 
#       proper collection of information for the files ID
#
#path=r'/Volumes/GoogleDrive/La meva unitat/Departament/Docència/Biocomputing Area/Interacció Bimolècules (3er)/2020-2021/test/'
path=r'/Volumes/GoogleDrive/La meva unitat/Departament/Docència/Biocomputing Area/Intro Bioinfo (2on)/2020-2021/test/'
ud(path)
#
# the file containing the students eith their 'First name', 'Surname' and 'Email address'
#students='courseid_22740_participants.csv'
students='courseid_22726_participants.csv'

#
# the file containing the questions, |-separated and with the fields 
# 'Points' | ' Questions' | 'data1' | 'data2' ...
# (rigth now only 'data1' is read, but extending it is straightforward)
#
test='testFINAL.csv'

#
# let us first create a couple of data frames with all the information
#
class_df = pd.read_csv(path+students).sort_values(by=['Surname'])
class_df = class_df.reset_index(drop=True)
test_df = pd.read_csv(path+test,sep='|',engine='python')
print(class_df)
print(test_df)

In [ ]:
import os, random

#
# the file that will contain the links to be sent to the students
#
links=path+'linksFINAL.txt'
fhl=open(links,'w')

mainInfo = """
How to answer this exam: You have 120 minutes to complete the test. Edit this file only with your answers, and 
do not share it with anyone. 

IMPORTANT: Edit this file using Google Docs, including screenshots to show your progress when needed. After 
finnishing the exam, create a PDF file and submit it to the assignment created in the Moodle course. You have 5 
file submission attempts. When the time will expire, the assignment will close and no further submissions 
will be possible. 
""".replace('\n','')
#
# let us iterate over each student, and then over the questions with their 'data1' specificities
# we will create a directory per student within 'path'
#

for ind in class_df.index:  
    personalID = class_df['Email address'][ind].split('@')[0]
    folder = path + '/FINAL_' + personalID
    file=folder+'/FINAL_'+personalID+'.txt'

    #
    # create the folder for each student in case it does not exist
    #
    if not os.path.exists(folder):
        os.makedirs(folder)
    fh=open(file,'w')
    who=class_df['First name'][ind]+' '+class_df['Surname'][ind]
    fh.write(who+'\n'+mainInfo+'\n\n\n')
    #
    # now iterate the exercises in the test
    #
    for ind2 in test_df.index:
        question=test_df['Question'][ind2].replace('\\n','\n')
        title='QUESTION '+str(ind2+1)+' ('+str(test_df['Points'][ind2])+' points):'+2*'\n'
        if 'XXX' in question:
            data=random.choice(str(test_df['data1'][ind2]).split(';'))
            #data=str(test_df['data1'][ind2]).split(';')[ind]
            question=question.replace('XXX',data)
        if 'YYY' in question:
            data=random.choice(str(test_df['data2'][ind2]).split(';'))
            #data=str(test_df['data2'][ind2]).split(';')[ind]
            question=question.replace('YYY',data)
        if 'ZZZ' in question:
            data=str(test_df['data3'][ind2]).split(';')[ind]
            question=question.replace('ZZZ',data)
        fh.write(title+str(question)+3*'\n') 
    fh.close()
    #
    # when using Google drive, it is good to get the info from its attributes
    #
    ID = !xattr -p user.drive.id '{folder}'
    infoLink = who+'\thttps://drive.google.com/drive/u/0/folders/'+ID.n
    fhl.write(infoLink+'\n')
    
    print(infoLink)
fhl.close()

ID = !xattr -p user.drive.id '{links}'
infoLink = '\thttps://drive.google.com/file/d/'+ID.n
print(links)
print(infoLink)